# 1. Xavier Initialization

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [9]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [4]:
# Dataset Loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [5]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [6]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0141, -0.0467, -0.0520,  ..., -0.1299, -0.0741, -0.1369],
        [-0.0814, -0.0076, -0.0670,  ..., -0.1282, -0.0473,  0.1394],
        [ 0.0297, -0.1215, -0.0284,  ...,  0.0223, -0.0105, -0.1363],
        ...,
        [-0.1378,  0.0897,  0.0736,  ..., -0.0051, -0.0927, -0.0670],
        [-0.0902, -0.1360, -0.0692,  ...,  0.1140, -0.1078,  0.0099],
        [ 0.1140, -0.1265,  0.0967,  ...,  0.1008,  0.0544, -0.0526]],
       requires_grad=True)

In [7]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3)

In [10]:
# define cost & optimizer
criterion = torch.nn.CrossEntropyLoss() # softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch: ', '%04d' % (epoch + 1), 'cost: ', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch:  0001 cost:  0.242849380
Epoch:  0002 cost:  0.092916958
Epoch:  0003 cost:  0.060861364
Epoch:  0004 cost:  0.043103602
Epoch:  0005 cost:  0.030785620
Epoch:  0006 cost:  0.024975641
Epoch:  0007 cost:  0.022686176
Epoch:  0008 cost:  0.018934205
Epoch:  0009 cost:  0.014009450
Epoch:  0010 cost:  0.015426308
Epoch:  0011 cost:  0.013425002
Epoch:  0012 cost:  0.008926275
Epoch:  0013 cost:  0.012268105
Epoch:  0014 cost:  0.008480563
Epoch:  0015 cost:  0.010340607
Learning finished


In [12]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Predcition: ', torch.argmax(single_prediction, 1).item())

Accuracy:  0.9793000221252441
Label:  1
Predcition:  1


/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/jjuke/opt/anaconda3/envs/DataMining/lib/python3.7/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
